<a href="https://colab.research.google.com/github/mihirrmbhaagwat/pythonprojects/blob/main/Data_engineering_and_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [183]:
import pandas as pd
import re
df = pd.read_csv("/content/edx_courses.csv")
print(" Data loaded successfully.")


 Data loaded successfully.


In [184]:
df

,title,summary,n_enrolled,course_type,institution,instructors,Level,subject,language,subtitles,avg_course_completion_per_week,course_effort,course_length_weeks,valid_certificate_price,price,course_description,course_syllabus,course_url
0,How to Learn Online,Learn essential strategies for successful onli...,"124,980",Self-paced on your time,edX,Nina Huntemann-Robyn Belair-Ben Piscopo,Introductory,Education & Teacher Training,English,English,2.5,2–3 hours per week,2,49,FREE-Add a Verified Certificate for $49 USD,"Designed for those who are new to elearning, t...",Welcome - We start with opportunities to meet ...,https://www.edx.org/course/how-to-learn-online
1,Programming for Everybody (Getting Started wit...,"This course is a ""no prerequisite"" introductio...","293,864",Self-paced on your time,The University of Michigan,Charles Severance,Introductory,Computer Science,English,English,3.0,2–4 hours per week,7,49,FREE-Add a Verified Certificate for $49 USD,This course aims to teach everyone the basics ...,NaN,https://www.edx.org/course/programming-for-eve...
2,CS50's Introduction to Computer Science,An introduction to the intellectual enterprise...,"2,442,271",Self-paced on your time,Harvard University,David J. Malan-Doug Lloyd-Brian Yu,Introductory,Computer Science,English,English,12.0,6–18 hours per week,12,90,FREE-Add a Verified Certificate for $90 USD,"This is CS50x , Harvard University's introduct...",NaN,https://www.edx.org/course/cs50s-introduction-...
3,The Analytics Edge,"Through inspiring examples and stories, discov...","129,555",Instructor-led on a course schedule,Massachusetts Institute of Technology,Dimitris Bertsimas-Allison O'Hair-John Silberh...,Intermediate,Data Analysis & Statistics,English,English,12.5,10–15 hours per week,13,199,FREE-Add a Verified Certificate for $199 USD,"In the last decade, the amount of data availab...",NaN,https://www.edx.org/course/the-analytics-edge
4,Marketing Analytics: Marketing Measurement Str...,This course is part of a MicroMasters® Program,"81,140",Self-paced on your time,"University of California, Berkeley",Stephan Sorger,Introductory,Computer Science,English,English,6.0,5–7 hours per week,4,249,FREE-Add a Verified Certificate for $249 USD,Begin your journey in a new career in marketin...,NaN,https://www.edx.org/course/marketing-analytics...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
970,Leaders in Citizen Security and Justice Manage...,"Learn about the latest in prevention, police a...",0,Self-paced on your time,Inter-American Development Bank,Olga Espinoza-Eduardo Pazinato-Alejandra Mera-...,Intermediate,Social Sciences,English,English,4.5,4–5 hours per week,10,25,FREE-Add a Verified Certificate for $25 USD,The high rates of crime and violence are two o...,NaN,https://www.edx.org/course/leaders-in-citizen-...
971,Pattern Studying and Making | 图案审美与创作,Fantastic experiences in beauty and its repres...,0,Self-paced on your time,Tsinghua University,Yuehua Nie,Introductory,Art & Culture,中文,"English, 中文",4.0,3–5 hours per week,12,139,FREE-Add a Verified Certificate for $139 USD,Are you an original designer? Or a DIY fancier...,NaN,https://www.edx.org/course/pattern-studying-an...
972,Computational Neuroscience: Neuronal Dynamics ...,This course explains the mathematical and comp...,"11,246",Self-paced on your time,École polytechnique fédérale de Lausanne,Wulfram Gerstner,Advanced,Biology & Life Sciences,English,English,5.0,4–6 hours per week,6,139,FREE-Add a Verified Certificate for $139 USD,What happens in your brain when you make a dec...,Textbook: Neuronal Dynamics - from single neur...,https://www.edx.org/course/computational-neuro...
973,Cities and the Challenge of Sustainable Develo...,What is a sustainable city? Learn the basics h...,"8,775",Self-paced on your time,SDG Academy,Jeffrey D. Sachs,Introductory,Environmental Studies,English,English,1.5,1–2 hours per week,1,25,FREE-Add a Verified Certificate for $25 USD,"According to the United Nat

In [185]:
import pandas as pd
import re

class EdxDataCleaner:
    def __init__(self, df):
        self.df = df.copy()

    def drop_unnecessary_columns(self):
      """
      Drops unwanted columns, case-insensitively.
      """
      cols_to_drop = [
          'summary', 'subtitles', 'course_effort',
          'course_description', 'course_syllabus', 'course_url'
      ]

      # Normalize both lists to lowercase for matching
      cols_in_df = self.df.columns
      matched_cols = [col for col in cols_in_df if col.lower() in cols_to_drop]

      self.df.drop(columns=matched_cols, inplace=True, errors='ignore')
      print(f" Dropped unnecessary columns: {matched_cols}")


    def standardize_column_names(self):
        self.df.columns = [(col[:1].upper() + col[1:]) if col else col for col in self.df.columns]
        print(" Standardized column names.")

    def rename_columns(self):
        rename_map = {
            'Title': 'Title',
            'N_enrolled': 'Enrollments',
            'Course_type': 'Course_Type',
            'Institution': 'Institution',
            'Level': 'Level',
            'Subject': 'Subject',
            'Language': 'Language',
            'Avg_course_completion_per_week': 'WeeklyHours',
            'Course_length_weeks': 'Duration(Wks)',
            'Valid_certificate_price': 'Certificate_Cost',
            'Price': 'Cost_of_Learning'
        }
        self.df.rename(columns=rename_map, inplace=True)
        print(" Renamed columns for better understanding.")

    def clean_course_type_and_enrollments(self):
        self.df['Course_Type'] = self.df['Course_Type'].map({
            'Self-paced on your time': 'Self-paced',
            'Instructor-led on a course schedule': 'Instructor-led'
        })
        self.df['Enrollments'] = self.df['Enrollments'].astype(str).str.replace(',', '').astype(int)
        print(" Cleaned Course_Type and Enrollments.")

    def create_price_tier(self):
        self.df['Certificate_Cost'] = self.df['Certificate_Cost'].astype(float)

        def assign_tier(price):
            if price == 0: return 'Free'
            elif price < 50: return 'Low'
            elif price < 100: return 'Medium'
            return 'High'

        self.df['PriceTier'] = self.df['Certificate_Cost'].apply(assign_tier)
        print(" Price tiers assigned.")


    def parse_uncertified_learning_cost(self):
        self.df['UncertifiedLearning_Cost'] = self.df['Cost_of_Learning'].str.upper().apply(
            lambda s: 0 if 'FREE' in s else 1
        )

        def _extract_currency(text):
            text = str(text)
            m = re.search(r'\b(USD|GBP|EUR|AUD|CAD)\b', text)
            if m:
                return m.group(1)
            m = re.search(r'([£$€])', text)
            if m:
                return m.group(1)
            return None

        self.df['Currency'] = self.df['Cost_of_Learning'].apply(_extract_currency)
        self.df.drop('Cost_of_Learning', axis=1, errors='ignore', inplace=True)
        print("Extracted uncertified cost flags and currency.")

    def cleaning_inspection(self):
      """
     Making sure everyting is clean

      """
      print("🧹 Full Cleaning Inspection Report:\n")
      for col in self.df.columns:
          print(f" Column: {col}")
          print(f"   - Missing values: {self.df[col].isna().sum()}")
          print(f"   - Dtype: {self.df[col].dtype}")
          print(f"   - Unique values: {self.df[col].nunique()}")
          print("")
      print(" Inspection complete.")

    def enforce_data_types(self):
      """
      Enforces correct data types on specific columns.
      Strings, integers, and floats as specified.
      """
      # Define target data types
      dtype_map = {
          # String columns
          'Currency': str,
          'PriceTier': str,
          'Level': str,
          'Language': str,
          'Subject': str,
          'Course_Type': str,
          'Institution': str,
          'Instructors': str,
          'Title': str,

          # Integer columns
          'Enrollments': 'Int64',   # Nullable int
          'Duration': 'Int64',

          # Float columns
          'WeeklyHours': 'float',
          'CertificateCost': 'float',
          'UncertifiedLearning_Cost': 'float'
      }

      # Apply conversions safely
      for col, dtype in dtype_map.items():
          if col in self.df.columns:
              try:
                  self.df[col] = self.df[col].astype(dtype)
              except Exception as e:
                  print(f"⚠️ Could not convert '{col}' to {dtype}: {e}")

      print(" Data types enforced.")


    def center_align_dataframe(self):

      """
      Returns a styled DataFrame with all text center-aligned.
      Useful for display in Jupyter/Colab.
      """
      return self.df.style\
          .set_properties(**{'text-align': 'center'})\
          .set_table_styles([
              {'selector': 'th', 'props': [('text-align', 'center')]}
          ])



    def run_all(self):
        self.standardize_column_names()
        self.rename_columns()             # Rename first
        self.drop_unnecessary_columns()   # Then drop renamed columns
        self.clean_course_type_and_enrollments()
        self.create_price_tier()
        self.parse_uncertified_learning_cost()
        self.cleaning_inspection()
        self.enforce_data_types()
        self.center_align_dataframe()
        print(" All cleaning steps completed.")
        return self.df


# Load your data



In [186]:
df = pd.read_csv("/content/edx_courses.csv")

# Create cleaner instance
cleaner = EdxDataCleaner(df)

# Run everything
cleaned_df = cleaner.run_all()

 Standardized column names.
 Renamed columns for better understanding.
 Dropped unnecessary columns: ['Summary', 'Subtitles', 'Course_effort', 'Course_description', 'Course_syllabus', 'Course_url']
 Cleaned Course_Type and Enrollments.
 Price tiers assigned.
Extracted uncertified cost flags and currency.
🧹 Full Cleaning Inspection Report:

 Column: Title
   - Missing values: 0
   - Dtype: object
   - Unique values: 972

 Column: Enrollments
   - Missing values: 0
   - Dtype: int64
   - Unique values: 850

 Column: Course_Type
   - Missing values: 0
   - Dtype: object
   - Unique values: 2

 Column: Institution
   - Missing values: 0
   - Dtype: object
   - Unique values: 111

 Column: Instructors
   - Missing values: 3
   - Dtype: object
   - Unique values: 775

 Column: Level
   - Missing values: 0
   - Dtype: object
   - Unique values: 3

 Column: Subject
   - Missing values: 0
   - Dtype: object
   - Unique values: 31

 Column: Language
   - Missing values: 0
   - Dtype: object
   -

In [187]:
df=cleaned_df


##Basic Data Analysis

In [188]:
top_institutions = (
    df.groupby('Institution')
    .agg(
        TotalEnrollments=('Enrollments', 'sum'),
        TotalCourses=('Title', 'count'),
        TotalInstructors=('Instructors', 'nunique')
    )
    .sort_values(by='TotalEnrollments', ascending=False)
    .head(10)
    .reset_index()
)

# Apply styling
styled_top = top_institutions.style.set_caption("Top 10 Institutions by Enrollment")\
    .format({
        'TotalEnrollments': '{:,.0f}',
        'TotalCourses': '{:,.0f}',
        'TotalInstructors': '{:,.0f}'
    })\
    .set_properties(**{'text-align': 'left'})\
    .set_table_styles([{
        'selector': 'th',
        'props': [('text-align', 'left')]
    }])

styled_top


,Institution,TotalEnrollments,TotalCourses,TotalInstructors
0,Harvard University,"13,459,914",104,63
1,The University of Queensland,"2,301,994",26,26
2,Universitat Politècnica de Valencia,"2,117,083",47,37
3,"University of California, Berkeley","1,840,798",20,6
4,Massachusetts Institute of Technology,"1,547,111",41,36
5,Columbia University,"1,341,270",20,16
6,Inter-American Development Bank,"1,115,101",29,26
7,Delft University of Technology,"1,073,553",30,30
8,Educational Testing Service,"1,022,489",1,1
9,The Linux Foundation,"1,001,977",10,10


In [189]:
# Step 1: Compute and format average certificate cost per institution
# Step 1: Compute average certificate cost
avg_cost_by_institution = (
    df.groupby('Institution')['Certificate_Cost']
    .mean()
    .round(2)
    .sort_values(ascending=False)
    .reset_index()
    .rename(columns={'Certificate_Cost': 'AvgCertificateCost'})
)

# Step 2: Extract Top 10 and Bottom 10
top_10 = avg_cost_by_institution.head(10).reset_index(drop=True)
bottom_10 = avg_cost_by_institution.tail(10).reset_index(drop=True)

# Step 3: Rename columns for clarity
top_10.columns = ['Top Institution', 'Top Avg Certificate Cost']
bottom_10.columns = ['Bottom Institution', 'Bottom Avg Certificate Cost']

# Step 4: Combine side-by-side
comparison_df = pd.concat([top_10, bottom_10], axis=1)

# Step 5: Style output (compatible with older pandas)
styled_output = comparison_df.style.set_caption("Top vs Bottom 10 Institutions by Avg Certificate Cost")\
    .format({
        'Top Avg Certificate Cost': '${:,.2f}',
        'Bottom Avg Certificate Cost': '${:,.2f}'
    })\
    .set_properties(**{'text-align': 'left'})\
    .set_table_styles([{
        'selector': 'th',
        'props': [('text-align', 'left')]
    }])

styled_output



,Top Institution,Top Avg Certificate Cost,Bottom Institution,Bottom Avg Certificate Cost
0,Boston University,$399.00,Universidades Anáhuac,$46.50
1,Western Governors University,$333.33,Catalyst,$44.60
2,University System of Maryland-University of Maryland Global Campus,$286.50,SDG Academy,$44.50
3,Chalmers University of Technology,$224.00,The Smithsonian Institution,$42.80
4,Rochester Institute of Technology,$220.64,Pontificia Universidad Javeriana,$40.60
5,New York University,$201.86,LOGYCA,$32.25
6,The National University of Singapore,$199.00,University of Baltimore-University System of Maryland,$29.00
7,The University of Michigan-Microsoft Education,$199.00,Inter-American Development Bank,$27.48
8,The Wharton School of the University of Pennsylvania,$199.00,The International Monetary Fund,$25.00
9,The World Wide Web Consortium (W3C),$199.00,World Bank Group,$5.00


In [190]:
# Ensure Enrollments and CertificateCost are numeric
df['Enrollments'] = pd.to_numeric(df['Enrollments'], errors='coerce')
df['Certificate_Cost'] = pd.to_numeric(df['Certificate_Cost'], errors='coerce')

# Drop rows where values are missing
df_clean = df.dropna(subset=['Enrollments', 'Certificate_Cost'])

# 1. Top 5 Courses by Highest Enrollment
top_enrollments = df_clean[['Title', 'Subject', 'Enrollments']]\
    .sort_values(by='Enrollments', ascending=False)\
    .head(5)

# 2. Bottom Courses by Lowest Enrollment

# 3. Top 5 Most Expensive Courses
top_cost = df_clean[['Title', 'Subject', 'Certificate_Cost']]\
    .sort_values(by='Certificate_Cost', ascending=False)\
    .head(5)

# 4. Bottom 5 Cheapest Courses (non-free)
bottom_cost = df_clean[df_clean['Certificate_Cost'] > 0][['Title', 'Subject', 'Certificate_Cost']]\
    .sort_values(by='Certificate_Cost', ascending=True)\
    .head(5)

# Style helper
def style_table(df_subset, caption, col_format=None):
    styler = df_subset.style.set_caption(caption)\
        .set_properties(**{'text-align': 'left'})\
        .set_table_styles([{'selector': 'th', 'props': [('text-align', 'left')]}])
    if col_format:
        styler = styler.format(col_format)
    return styler

# Display styled tables
styled_top_enrollments = style_table(top_enrollments, "🔝 Top 5 Courses by Enrollment", {'Enrollments': '{:,.0f}'})
styled_bottom_enrollments = style_table(bottom_enrollments, "🔻 Bottom 5 Courses by Enrollment", {'Enrollments': '{:,.0f}'})
styled_top_cost = style_table(top_cost, "💰 Top 5 Most Expensive Courses", {'CertificateCost': '${:,.2f}'})
styled_bottom_cost = style_table(bottom_cost, "💸 Bottom 5 Cheapest Courses", {'CertificateCost': '${:,.2f}'})

# Return all 4 styled tables (not print)
styled_top_enrollments



,Title,Subject,Enrollments
2,CS50's Introduction to Computer Science,Computer Science,"2,442,271"
29,IELTS Academic Test Preparation,Communication,"1,103,777"
31,TOEFL® Test Preparation: The Insider’s Guide,Communication,"1,022,489"
23,CS50's Web Programming with Python and JavaScript,Computer Science,"698,950"
64,Introduction to Linux,Computer Science,"642,088"


In [191]:
# 1. Filter courses with 0 enrollments
zero_enrollments_df = df_clean[df_clean['Enrollments'] == 0]

# 2. Count how many
zero_enrollments_count = zero_enrollments_df.shape[0]
print(f"📉 Total courses with 0 enrollments: {zero_enrollments_count}")

# 3. Subset relevant columns
zero_enrollments_display = zero_enrollments_df[['Title', 'Subject']].reset_index(drop=True)

# 4. Style the table
styled_zero_enrollments = zero_enrollments_display.style\
    .set_caption("📋 Courses with 0 Enrollments")\
    .set_properties(**{'text-align': 'left'})\
    .set_table_styles([{
        'selector': 'th',
        'props': [('text-align', 'left')]
    }])

styled_zero_enrollments




📉 Total courses with 0 enrollments: 120


,Title,Subject
0,English Grammar and Style,Humanities
1,Business Communication,Communication
2,Agua 2.0: empresas eficientes para el siglo XXI,Business & Management
3,Introduction to Cloud Computing,Computer Science
4,SQL aplicado en la ciencia de datos,Data Analysis & Statistics
5,Sign Language Science: Factors Contributing to Nature Structure,Language
6,Gender and Intersectionality,Humanities
7,"MathTrackX: Polynomials, Functions and Graphs",Math
8,Negotiation Strategies and Styles,Business & Management
9,Introducción a la Inteligencia Artificial de Watson,Computer Science


In [192]:
styled_top_cost


,Title,Subject,Certificate_Cost
157,Foundations of Modern Finance I,Economics & Finance,450.000000
543,Information Technology Foundations,Computer Science,400.000000
19,Strategic Social Media Marketing,Business & Management,399.000000
842,Driving Digital Innovation through Experimentation,Computer Science,399.000000
594,Energy Principles and Renewable Energy,Energy & Earth Sciences,375.000000


In [193]:
styled_bottom_cost

,Title,Subject,Certificate_Cost
466,e-Learning on Digital Agriculture,Environmental Studies,5.000000
417,Solid Waste Management,Environmental Studies,5.000000
458,Unlocking Investment and Finance in Emerging Markets and Developing Economies (EMDEs),Economics & Finance,5.000000
564,The Future of Work: Preparing for Disruption,Economics & Finance,5.000000
136,Design Thinking and Creativity for Innovation,Business & Management,5.000000


In [194]:
print(df['Course_Type'].unique())
print(df['Course_Type'].value_counts())


['Self-paced' 'Instructor-led']
Course_Type
Self-paced        917
Instructor-led     58
Name: count, dtype: int64
